# Carrier Invoice Data Analysis & CSV Export

This notebook provides tools for querying and exporting carrier invoice data from ClickHouse and DuckDB to CSV files.

## 📋 What's Available:
- **ClickHouse Direct Export** - Export full dataset directly from ClickHouse
- **DuckDB Export** - Export already extracted data from DuckDB
- **Batch Processing** - Handle large datasets efficiently
- **Multiple Export Options** - Sample, test, and full exports

Let's start! 🚀

In [8]:
# Import required libraries
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ClickHouse and DuckDB imports
try:
    import clickhouse_connect
    CLICKHOUSE_AVAILABLE = True
    print("✅ ClickHouse library available")
except ImportError:
    CLICKHOUSE_AVAILABLE = False
    print("❌ ClickHouse library not available")

try:
    import duckdb
    DUCKDB_AVAILABLE = True
    print("✅ DuckDB library available")
except ImportError:
    DUCKDB_AVAILABLE = False
    print("❌ DuckDB library not available")

print(f"\n📊 Pandas version: {pd.__version__}")
print(f"✅ All libraries imported successfully!")

✅ ClickHouse library available
✅ DuckDB library available

📊 Pandas version: 2.3.1
✅ All libraries imported successfully!


In [9]:
# Setup for data export
# Create output directory if it doesn't exist
output_dir = "../output"
os.makedirs(output_dir, exist_ok=True)

# Generate timestamp for file naming
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

print(f"📁 Output directory: {os.path.abspath(output_dir)}")
print(f"🕒 Timestamp for files: {timestamp}")
print(f"✅ Ready to export data!")

📁 Output directory: c:\Users\Gabriel\Documents\gsr_project\gsr_automation\data\output
🕒 Timestamp for files: 20250807_214415
✅ Ready to export data!


## 🚀 Direct ClickHouse to CSV Export

Export the complete carrier invoice dataset directly from ClickHouse to CSV files.

In [10]:
# Export SAMPLE dataset from ClickHouse (100k records)
print("🧪 Sample Export from ClickHouse (100k records)")
print("=" * 50)

if not CLICKHOUSE_AVAILABLE:
    print("❌ ClickHouse library not available. Install with: pip install clickhouse-connect")
else:
    try:
        print("📡 Connecting to ClickHouse...")
        
        # Connect to ClickHouse
        ch_client = clickhouse_connect.get_client(
            host="pgy8egpix3.us-east-1.aws.clickhouse.cloud",
            port=8443,
            username="gabriellapuz",
            password="PTN.776)RR3s",
            database="peerdb",
            secure=True,
        )
        
        print("✅ Connected to ClickHouse successfully!")
        
        # Get total record count
        table_name = "carrier_carrier_invoice_original_flat_ups"
        total_count = ch_client.command(f"SELECT COUNT(*) FROM {table_name}")
        print(f"📊 Total records in ClickHouse: {total_count:,}")
        
        # Export sample (100k records)
        sample_size = 100000
        print(f"\n🔄 Exporting sample ({sample_size:,} records)...")
        
        # Query sample data
        sample_query = f"""
            SELECT * FROM {table_name} 
            ORDER BY import_time DESC 
            LIMIT {sample_size}
        """
        
        print("🔄 Executing ClickHouse query...")
        result = ch_client.query(sample_query)
        
        # Convert to DataFrame
        print("📊 Converting to DataFrame...")
        df = result.df()
        
        # Export to CSV
        csv_filename = f"carrier_invoice_sample_{sample_size}_{timestamp}.csv"
        csv_path = os.path.join(output_dir, csv_filename)
        
        print(f"💾 Saving to CSV: {csv_filename}")
        df.to_csv(csv_path, index=False)
        
        # Get file info
        file_size = os.path.getsize(csv_path) / (1024 * 1024)  # MB
        
        print(f"\n✅ Sample export completed successfully!")
        print(f"   📄 File: {csv_filename}")
        print(f"   📊 Records: {len(df):,}")
        print(f"   📋 Columns: {len(df.columns)}")
        print(f"   💾 Size: {file_size:.2f} MB")
        print(f"   📁 Location: {os.path.abspath(csv_path)}")
        
        # Close connection
        ch_client.close()
        
    except Exception as e:
        print(f"❌ Sample export failed: {e}")
        print("💡 Check your ClickHouse connection and try again")

🧪 Sample Export from ClickHouse (100k records)
📡 Connecting to ClickHouse...
✅ Connected to ClickHouse successfully!
📊 Total records in ClickHouse: 13,424,252

🔄 Exporting sample (100,000 records)...
🔄 Executing ClickHouse query...
❌ Sample export failed: HTTPDriver for https://pgy8egpix3.us-east-1.aws.clickhouse.cloud:8443 received ClickHouse error code 241
 Code: 241. DB::Exception: (total) memory limit exceeded: would use 10.80 GiB (attempt to allocate chunk of 4.00 MiB bytes), current RSS: 10.80 GiB, maximum: 10.80 GiB. OvercommitTracker decision: Query was selected to stop by OvercommitTracker: While executing MergeSortingTransform. (MEMORY_LIMIT_EXCEEDED) (version 25.6.2.5781 (official build))

💡 Check your ClickHouse connection and try again


In [11]:
# Batch export for large datasets - exports in chunks to manage memory
print("📦 Batch Export from ClickHouse (Recommended for Large Datasets)")
print("=" * 65)

def export_clickhouse_in_batches(batch_size=50000, max_batches=None):
    """
    Export ClickHouse data in batches to manage memory usage
    
    Args:
        batch_size (int): Number of records per batch
        max_batches (int): Maximum number of batches (None = all data)
    """
    if not CLICKHOUSE_AVAILABLE:
        print("❌ ClickHouse library not available")
        return None
        
    try:
        print("📡 Connecting to ClickHouse...")
        
        # Connect to ClickHouse
        ch_client = clickhouse_connect.get_client(
            host="pgy8egpix3.us-east-1.aws.clickhouse.cloud",
            port=8443,
            username="gabriellapuz",
            password="PTN.776)RR3s",
            database="peerdb",
            secure=True,
        )
        
        table_name = "carrier_carrier_invoice_original_flat_ups"
        
        # Get total count
        total_count = ch_client.command(f"SELECT COUNT(*) FROM {table_name}")
        total_batches = (total_count + batch_size - 1) // batch_size
        
        if max_batches:
            total_batches = min(total_batches, max_batches)
            total_records = min(total_count, max_batches * batch_size)
        else:
            total_records = total_count
        
        print(f"📊 Total records: {total_count:,}")
        print(f"📦 Batch size: {batch_size:,}")
        print(f"🔢 Total batches to export: {total_batches}")
        print(f"📈 Records to export: {total_records:,}")
        
        # Create main CSV file
        csv_filename = f"carrier_invoice_full_{timestamp}.csv"
        csv_path = os.path.join(output_dir, csv_filename)
        
        print(f"\n🚀 Starting batch export...")
        
        first_batch = True
        total_exported = 0
        
        for batch_num in range(total_batches):
            offset = batch_num * batch_size
            
            print(f"📦 Processing batch {batch_num + 1}/{total_batches} (offset: {offset:,})...")
            
            # Query batch
            batch_query = f"""
                SELECT * FROM {table_name}
                ORDER BY import_time DESC
                LIMIT {batch_size} OFFSET {offset}
            """
            
            result = ch_client.query(batch_query)
            df_batch = result.df()
            
            if df_batch.empty:
                print(f"   ℹ️ No more data, stopping at batch {batch_num + 1}")
                break
            
            # Append to CSV (write header only for first batch)
            df_batch.to_csv(csv_path, mode='a', header=first_batch, index=False)
            
            total_exported += len(df_batch)
            first_batch = False
            
            print(f"   ✅ Batch {batch_num + 1} completed: {len(df_batch):,} records")
            print(f"   📊 Total exported so far: {total_exported:,}")
            
            # Show progress
            progress = (batch_num + 1) / total_batches * 100
            print(f"   📈 Progress: {progress:.1f}%")
        
        # Final file info
        file_size = os.path.getsize(csv_path) / (1024 * 1024)  # MB
        
        print(f"\n🎉 Batch export completed successfully!")
        print(f"   📄 File: {csv_filename}")
        print(f"   📊 Total records exported: {total_exported:,}")
        print(f"   💾 File size: {file_size:.2f} MB")
        print(f"   📁 Location: {os.path.abspath(csv_path)}")
        
        ch_client.close()
        return csv_path
        
    except Exception as e:
        print(f"❌ Batch export failed: {e}")
        return None

# Example usage - export first 5 batches (250k records) as a test
print("🧪 Test run: Exporting first 5 batches (250k records)")
print("💡 Change max_batches=None to export ALL data")

result_path = export_clickhouse_in_batches(batch_size=50000, max_batches=5)

if result_path:
    print(f"\n✅ Test export successful!")
    print(f"💡 To export ALL data, run: export_clickhouse_in_batches(batch_size=50000, max_batches=None)")
else:
    print(f"\n❌ Test export failed")

📦 Batch Export from ClickHouse (Recommended for Large Datasets)
🧪 Test run: Exporting first 5 batches (250k records)
💡 Change max_batches=None to export ALL data
📡 Connecting to ClickHouse...
📊 Total records: 13,424,252
📦 Batch size: 50,000
🔢 Total batches to export: 5
📈 Records to export: 250,000

🚀 Starting batch export...
📦 Processing batch 1/5 (offset: 0)...
❌ Batch export failed: HTTPDriver for https://pgy8egpix3.us-east-1.aws.clickhouse.cloud:8443 received ClickHouse error code 241
 Code: 241. DB::Exception: (total) memory limit exceeded: would use 8.12 GiB (attempt to allocate chunk of 4.01 MiB bytes), current RSS: 10.87 GiB, maximum: 10.80 GiB. OvercommitTracker decision: Query was selected to stop by OvercommitTracker: While executing MergeSortingTransform. (MEMORY_LIMIT_EXCEEDED) (version 25.6.2.5781 (official build))


❌ Test export failed


In [12]:
# FULL EXPORT - Run this cell to export ALL 13+ million records
print("🚨 FULL DATASET EXPORT - ALL 13+ MILLION RECORDS")
print("=" * 55)
print("⚠️ WARNING: This will take 20-60 minutes and create a very large file (several GB)")
print("💡 Make sure you have enough disk space and a stable connection")
print("\n🔄 Uncomment the line below to start the full export:")
print("\n# result_path = export_clickhouse_in_batches(batch_size=100000, max_batches=None)")

# Uncomment the line below to run the full export
# result_path = export_clickhouse_in_batches(batch_size=100000, max_batches=None)

print("\n💡 To run the full export:")
print("   1. Uncomment the line above")
print("   2. Run this cell")
print("   3. Wait for completion (20-60 minutes)")
print("   4. Check data/output/ folder for the CSV file")

🚨 FULL DATASET EXPORT - ALL 13+ MILLION RECORDS
⚠️ WARNING: This will take 20-60 minutes and create a very large file (several GB)
💡 Make sure you have enough disk space and a stable connection

🔄 Uncomment the line below to start the full export:

# result_path = export_clickhouse_in_batches(batch_size=100000, max_batches=None)

💡 To run the full export:
   1. Uncomment the line above
   2. Run this cell
   3. Wait for completion (20-60 minutes)
   4. Check data/output/ folder for the CSV file


## 🦆 DuckDB to CSV Export

Export data that was already extracted by the pipeline from DuckDB to CSV.

In [13]:
# Export data from DuckDB file (if it exists)
print("🦆 DuckDB to CSV Export")
print("=" * 25)

if not DUCKDB_AVAILABLE:
    print("❌ DuckDB library not available")
else:
    try:
        # Connect to the DuckDB file created by the pipeline
        db_path = "../../carrier_invoice_extraction.duckdb"
        
        if not os.path.exists(db_path):
            print(f"❌ DuckDB file not found: {db_path}")
            print("💡 Run the pipeline first: poetry run python src/src/dlt_pipeline_examples.py")
        else:
            conn = duckdb.connect(db_path)
            print(f"✅ Connected to DuckDB: {db_path}")
            
            # Check the table
            table_name = "carrier_invoice_data.carrier_invoice_data"
            count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
            print(f"📊 Records in DuckDB: {count:,}")
            
            # Export to CSV
            print("🔄 Exporting DuckDB data to CSV...")
            
            # Get all data
            df = conn.execute(f"SELECT * FROM {table_name}").df()
            
            # Export to CSV
            csv_filename = f"carrier_invoice_duckdb_{timestamp}.csv"
            csv_path = os.path.join(output_dir, csv_filename)
            
            df.to_csv(csv_path, index=False)
            
            # Get file info
            file_size = os.path.getsize(csv_path) / (1024 * 1024)  # MB
            
            print(f"\n✅ DuckDB export completed!")
            print(f"   📄 File: {csv_filename}")
            print(f"   📊 Records: {len(df):,}")
            print(f"   📋 Columns: {len(df.columns)}")
            print(f"   💾 Size: {file_size:.2f} MB")
            print(f"   📁 Location: {os.path.abspath(csv_path)}")
            
            conn.close()
            
    except Exception as e:
        print(f"❌ DuckDB export failed: {e}")

🦆 DuckDB to CSV Export
✅ Connected to DuckDB: ../../carrier_invoice_extraction.duckdb
📊 Records in DuckDB: 50
🔄 Exporting DuckDB data to CSV...

✅ DuckDB export completed!
   📄 File: carrier_invoice_duckdb_20250807_214415.csv
   📊 Records: 50
   📋 Columns: 263
   💾 Size: 0.06 MB
   📁 Location: c:\Users\Gabriel\Documents\gsr_project\gsr_automation\data\output\carrier_invoice_duckdb_20250807_214415.csv


In [14]:
# List all exported files
print("📁 Exported Files Summary")
print("=" * 30)

try:
    # Get all CSV files in output directory
    if os.path.exists(output_dir):
        csv_files = [f for f in os.listdir(output_dir) if f.endswith('.csv')]
        
        if csv_files:
            print(f"📊 Found {len(csv_files)} CSV files in output directory:")
            print()
            
            total_size = 0
            for i, filename in enumerate(sorted(csv_files), 1):
                filepath = os.path.join(output_dir, filename)
                file_size = os.path.getsize(filepath) / (1024 * 1024)  # MB
                total_size += file_size
                
                print(f"   {i}. {filename}")
                print(f"      💾 Size: {file_size:.2f} MB")
                print(f"      📁 Path: {os.path.abspath(filepath)}")
                print()
            
            print(f"📊 Total exported data: {total_size:.2f} MB")
            print(f"📁 Output directory: {os.path.abspath(output_dir)}")
        else:
            print(f"ℹ️ No CSV files found in output directory")
            print(f"💡 Run the export cells above to create CSV files")
    else:
        print(f"❌ Output directory does not exist: {output_dir}")
        
except Exception as e:
    print(f"❌ Error listing files: {e}")

📁 Exported Files Summary
📊 Found 1 CSV files in output directory:

   1. carrier_invoice_duckdb_20250807_214415.csv
      💾 Size: 0.06 MB
      📁 Path: c:\Users\Gabriel\Documents\gsr_project\gsr_automation\data\output\carrier_invoice_duckdb_20250807_214415.csv

📊 Total exported data: 0.06 MB
📁 Output directory: c:\Users\Gabriel\Documents\gsr_project\gsr_automation\data\output
